参考：[【連載】LangChainの公式チュートリアルを1個ずつ地味に､地道にコツコツと【Basic編#1】](https://zenn.dev/chips0711/articles/f4ed8ac37eb3a8)

### 環境設定
※ライブラリはlangchain v0.2系ではなく最新のv0.3系を使用したため参考サイトとはバージョンが異なる

In [15]:
!python -V
# !pip install ipykernel
# !pip install langchain-openai==0.2.14 langchain==0.3.13 langserve==0.3.0 python-dotenv
!pip list

Python 3.10.7
Package                  Version
------------------------ -----------
aiohappyeyeballs         2.4.4
aiohttp                  3.11.11
aiosignal                1.3.2
annotated-types          0.7.0
anyio                    4.7.0
asttokens                3.0.0
async-timeout            4.0.3
attrs                    24.3.0
certifi                  2024.12.14
charset-normalizer       3.4.1
click                    8.1.8
colorama                 0.4.6
comm                     0.2.2
debugpy                  1.8.11
decorator                5.1.1
distro                   1.9.0
exceptiongroup           1.2.2
executing                2.1.0
fastapi                  0.115.6
frozenlist               1.5.0
greenlet                 3.1.1
h11                      0.14.0
httpcore                 1.0.7
httpx                    0.27.2
idna                     3.10
ipykernel                6.29.5
ipython                  8.31.0
jedi                     0.19.2
jiter                    0.8.2
js

### 英語から日本語への翻訳
17行目でcontentに指定した英語テキストを日本語に翻訳する  
例：Hi! → こんにちは！

In [3]:
import os
import json

from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv())

# Azure OpenAIのGPT-4o miniモデルを使用してAzureChatOpenAIインスタンスを作成します。
model = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
)

# 翻訳指示とユーザーからのメッセージを定義します。
messages = [
    SystemMessage(content="Translate the following from English into Japanese"),
    HumanMessage(content="Hi!"),
]

# # モデルにメッセージを渡して翻訳を実行します。
response = model.invoke(messages)
formated_response = json.dumps(response.__dict__, indent=4, ensure_ascii=False)
print(formated_response)

{
    "content": "こんにちは！",
    "additional_kwargs": {
        "refusal": null
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 2,
            "prompt_tokens": 20,
            "total_tokens": 22,
            "completion_tokens_details": null,
            "prompt_tokens_details": null
        },
        "model_name": "gpt-4o-mini-2024-07-18",
        "system_fingerprint": "fp_04751d0b65",
        "prompt_filter_results": [
            {
                "prompt_index": 0,
                "content_filter_results": {
                    "hate": {
                        "filtered": false,
                        "severity": "safe"
                    },
                    "jailbreak": {
                        "filtered": false,
                        "detected": false
                    },
                    "self_harm": {
                        "filtered": false,
                        "severity": "safe"
                    },
             

### OutputParserによるレスポンスの整形

In [8]:
from langchain_core.output_parsers import StrOutputParser

# 出力を文字列として取得するためのOutputParserをインスタンス化します。
parser = StrOutputParser()

# モデルからのレスポンスを文字列としてパースします。
parsed_result = parser.invoke(response)
print(parsed_result)

こんにちは！


### PromptTemplatesの導入
動的なプロンプトの生成

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# システムメッセージのテンプレートを定義します。
system_template = "Translate the following into {language}:"

# ユーザーからの入力とシステムメッセージを基にプロンプトテンプレートを作成します。
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

# テンプレートを使ってプロンプトをフォーマットします。
input_data = {"language": "Japanese", "text": "Hi!"}
formatted_prompt = prompt_template.invoke(input_data)
print(formatted_prompt)

messages=[SystemMessage(content='Translate the following into Japanese:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={})]


### LangChain Expression Language (LCEL)
LangChainのコンポーネントを宣言的にチェーンするための方法  

##### LCELの主な特徴
 - ストリーミングサポート
 - 非同期サポート
 - 最適化された並列実行
 - リトライとフォールバック
 - 中間結果へのアクセス
 - 入力と出力のスキーマ
 - LangSmithとのシームレスなトレース

#### LCELを使ったコンポーネントのチェーン化
モデルの呼び出しから出力の取得までを一つの流れで処理できる

In [9]:
# プロンプトテンプレート、モデル、出力パーサーを組み合わせてチェーンを作成します。
chain = prompt_template | model | parser

# チェーンに入力を渡して結果を取得します。
result = chain.invoke({"language": "Japanese", "text": "Hi!"})
print(result)

こんにちは！


### LangServeを使ったアプリケーションのデプロイ
 - LangServeを使用すると、LangChainのチェーンをREST APIとして簡単に提供できる  
 - LangServeは、FastAPIと統合されており、Pydanticを使用したデータバリデーションもサポートしている  
 - これにより、エンドポイントの入力と出力スキーマが自動的に推論され、すべてのAPIコールで強制されるため、エラーメッセージが豊富になる

#### サーバーのセットアップ

##### ライブラリの追加

In [ ]:
# サーバー用
#!pip install fastapi==0.115.6 uvicorn==0.34.0
# バージョン関連の問題解決用
#!httpx==0.27.2 starlette==0.41.3

##### serve.pyを作成してサーバを起動しておく
 - [サーバーリンク](http://localhost:8000/chain)  
 - [APIの仕様やエンドポイントの詳細リンク](http://localhost:8000/docs)

In [ ]:
"""serve.py
保存したディレクトリで「python serve.py」コマンドを実行
"""
import os

from dotenv import find_dotenv, load_dotenv
from fastapi import FastAPI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langserve import add_routes

load_dotenv(find_dotenv())

# プロンプトテンプレート、モデル、出力パーサーを定義します。
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([("system", system_template), ("user", "{text}")])
model = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
)
parser = StrOutputParser()

# チェーンを定義します。
chain = prompt_template | model | parser

# appの定義としてFastAPIアプリケーションのインスタンスを作成します。
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple API server using LangChain's Runnable interfaces",
)

# チェーンをエンドポイントに追加します。
add_routes(app, chain, path="/chain")

if __name__ == "__main__":
    import uvicorn

    # サーバーを起動します。
    uvicorn.run(app, host="localhost", port=8000)

#### アプリケーションの利用例

##### requestsライブラリでの例
 - Pythonのrequestsライブラリを使用して、エンドポイントにリクエストを送信

In [13]:
import requests

# APIエンドポイントに送信するデータ
data = {"language": "Japanese", "text": "Today is a beatuiful day."}

# POSTリクエストを送信して結果を取得
response = requests.post("http://localhost:8000/chain/invoke", json={'input': data})

# エラーチェック
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


{'output': '今日は美しい日です。', 'metadata': {'run_id': '98192bbe-b728-4754-b080-5a45223b5ba6', 'feedback_tokens': []}}


##### langserve.RemoteRunnableクライアントでの例
 - langserveのRemoteRunnableを使用して、エンドポイントにリクエストを送信  
→ httpxライブラリの問題？で動作せず・・・

In [23]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "Japanese", "text": "Today is a beautiful day."})

ImportError: cannot import name 'VerifyTypes' from 'httpx._types' (c:\Users\naohiro\Desktop\desktop_memo\program\LangChain_tutorial\.venv\lib\site-packages\httpx\_types.py)

#### LangServe組み込みのUIでの例
 - [WEB UI](http://localhost:8000/chain/playground/)からのAPI実行も可能